---
title: "Reproducing Cui Xiang-Han et. al (2020)'s 'Fast radio bursts: do repeaters and non-repeaters originate in statistically similar ensembles?'"
subtitle: Is it possible that repeaters and non-repeaters have different mechanisms?
author: Murthadza Aznam
date: '2022-12-26'

---

:::{.callout-note}

 📌 Goal: This notebook tries to reproduce the results from [https://ui.adsabs.harvard.edu/abs/2021MNRAS.500.3275C/abstract](https://ui.adsabs.harvard.edu/abs/2021MNRAS.500.3275C/abstract). As stated in the paper, verbatim 
 
 > We investigate the statistical classifications for the two groups of samples **to see if the non-repeating and repeating FRBs have different origins** by employing Anderson–Darling (A–D) test and Mann–WhitneyWilcoxon (M–W–W) test

:::

In [1]:
import pandas as pd
import seaborn as sns

## 0. Getting The Data
The paper uses 111 samples (20 repeating + 91 non-repeating) of FRB.

### 0.1 Source
The data underlying this article are available in the references below: 
    1. Spitler et al. ([2016](https://www.nature.com/articles/nature17168)); 
    2. CHIME/FRB Collaboration ([2019a](https://www.nature.com/articles/s41586-018-0864-x));
    3. Kumar et al. ([2019](https://iopscience.iop.org/article/10.3847/2041-8213/ab5b08));
    4. CHIME/FRB Collaboration ([2019b](https://iopscience.iop.org/article/10.3847/2041-8213/ab4a80));
    5. Fonseca et al. ([2020](https://iopscience.iop.org/article/10.3847/2041-8213/ab7208)). 
Some data of FRBs are taken from the data base of FRBCAT, available at http://www.frbcat.org/.